In [1]:
import os
import pandas as pd
import xlrd
import time
from deep_translator import GoogleTranslator
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

### 1) Uploading dataset and translating
1.1) Loading the already cleaned OECD dataset

In [2]:
df = pd.read_excel("OECD_All_v3.xlsx", sheet_name = "All")

1.2) Substituting all blank values (NaN) in "Description" and "Project Title" with "hello world"

In [3]:
df["Description"] = df["Description"].fillna("hello_world")
df["Project Title"] = df["Project Title"].fillna("hello_world")

1.3) Dropping project ID numbers and separating the labels (dependant variables) from the fields (independent variables)

In [6]:
df = df.drop(['CRS Identification N°', 'Donor project N°'], axis = 1)

In [7]:
labels_df = df[["Climate objective (applies to Rio-marked data only) or climate component", 
              "Adaptation objective (applies to Rio-marked data only)", 
              "Mitigation objective (applies to Rio-marked data only)"]]

In [8]:
df = df.drop(["Climate objective (applies to Rio-marked data only) or climate component", 
              "Adaptation objective (applies to Rio-marked data only)", 
              "Mitigation objective (applies to Rio-marked data only)"], axis = 1)

1.4) Translating all non-english entries in "Description" and "Project title" to English.

If the "Description" and "Project title" fields have already been translated, upload the results:

In [4]:
with open("title_desc_167363.txt", "r+", encoding="utf-8") as td_file:    
    td_content = td_file.read()

In [5]:
title_desc_en = td_content.split("_$%$&_")

In [11]:
#df_title_desc = pd.DataFrame({"Title_Desc" : title_desc_en })

If the "Description" and "Project title" fields have not been translated yet, follow this procedure:

In [12]:
#ptitle = df["Project Title"].astype(str).tolist()
#desc = df["Description"].astype(str).tolist()

In [13]:
#title_desc = list(map(lambda x,y: x + " /// " + y, ptitle, desc))

In [14]:
#title_desc_en = []
#for j in range(115503, len(title_desc)):
    #td_en = GoogleTranslator('auto', 'english').translate(title_desc[j])
    #title_desc_en.append(td_en)

#### 2) Encoding the fields and the Rio labels:

2.1) In the 4 funds related fields, if a project shows values for 3 of them, the 4th can be estimated out of them.

In [15]:
funds_fields = ['Adaptation-related development finance - Commitment - 2021 USD thousand',
       'Mitigation-related development finance - Commitment - 2021 USD thousand',
       'Overlap - Commitment - 2021 USD thousand',
       'Climate-related development finance - Commitment - 2021 USD thousand']

In [16]:
df["Adaptation_Commitment"] = df['Climate-related development finance - Commitment - 2021 USD thousand']
-df['Mitigation-related development finance - Commitment - 2021 USD thousand']
+df['Overlap - Commitment - 2021 USD thousand']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
167358    0.0
167359    0.0
167360    0.0
167361    0.0
167362    0.0
Name: Overlap - Commitment - 2021 USD thousand, Length: 167363, dtype: float64

In [17]:
df["Mitigation_Commitment"] = df['Climate-related development finance - Commitment - 2021 USD thousand']
-df['Adaptation-related development finance - Commitment - 2021 USD thousand']
+df['Overlap - Commitment - 2021 USD thousand']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
167358    0.0
167359    0.0
167360    0.0
167361    0.0
167362    0.0
Name: Overlap - Commitment - 2021 USD thousand, Length: 167363, dtype: float64

In [18]:
df["Overlap_Commitment"] = df['Adaptation-related development finance - Commitment - 2021 USD thousand']
+df['Mitigation-related development finance - Commitment - 2021 USD thousand']
-df['Climate-related development finance - Commitment - 2021 USD thousand']

0           -25.828146
1           -40.781284
2           -14.148360
3         -5454.178613
4        -12251.975464
              ...     
167358    -6149.025009
167359      -49.020000
167360   -31579.170000
167361    -6000.000000
167362   -29501.000000
Name: Climate-related development finance - Commitment - 2021 USD thousand, Length: 167363, dtype: float64

In [19]:
df['Adaptation-related development finance - Commitment - 2021 USD thousand'].fillna(df["Adaptation_Commitment"], inplace = True)

In [20]:
df['Mitigation-related development finance - Commitment - 2021 USD thousand'].fillna(df["Mitigation_Commitment"], inplace = True)

In [21]:
df['Overlap - Commitment - 2021 USD thousand'].fillna(df["Overlap_Commitment"], inplace = True)

In [22]:
df.drop(["Adaptation_Commitment", "Mitigation_Commitment", "Overlap_Commitment"], axis = 1, inplace = True)

2.2) Substitute blank values (NaN) with 0s and -1s.

In [23]:
funds_fields = ['Adaptation-related development finance - Commitment - 2021 USD thousand',
       'Mitigation-related development finance - Commitment - 2021 USD thousand',
       'Overlap - Commitment - 2021 USD thousand',
       'Climate-related development finance - Commitment - 2021 USD thousand']

In [24]:
no_funds_fields = [field for field in df.columns if field not in funds_fields and field not in ['Project Title', 'Description']]

In [25]:
non_numeric_fields = ['Provider Type', 'Provider', 'Provider (detailed)', 'Extending Agency', 'Recipient Region', 'Recipient Income Group (OECD Classification)',  'Concessionality', 'Channel of Delivery', 'Sector (detailed)',
 'Sub-sector', 'Development Cooperation Modality', 'Financial Instrument', 'Gender']

In [26]:
numeric_fields = [field for field in no_funds_fields if field not in non_numeric_fields]

In [27]:
df[no_funds_fields] = df[no_funds_fields].fillna(0)

In [28]:
df[funds_fields] = df[funds_fields].fillna(-1)

In [29]:
#Check
print(df[no_funds_fields].isna().sum())
print(df[funds_fields].isna().sum())

Year                                            0
Provider Type                                   0
Provider                                        0
Provider (detailed)                             0
Provider Code                                   0
Agency Code                                     0
Extending Agency                                0
Recipient Code                                  0
Recipient Region                                0
Recipient Income Group (OECD Classification)    0
Concessionality                                 0
Channel of Delivery Code                        0
Channel of Delivery                             0
Purpose Code                                    0
Sector (detailed)                               0
Sub-sector                                      0
Development Cooperation Modality                0
Financial Instrument                            0
Type of Finance                                 0
Coal-related Financing                          0


2.3) Created dataframe for funding-related fields (Case A in report)

In [30]:
funds_df = df[funds_fields]

In [31]:
funds_df.to_csv('encoded_case_a_0.csv', index=True)

2.3) Encoding non-numeric fields (Case C in report).

In [32]:
nonnum_df = df[non_numeric_fields].astype(str)

In [33]:
enc_nonnum = OneHotEncoder(handle_unknown='ignore')

In [34]:
enc_nonnum.fit(nonnum_df)

OneHotEncoder(handle_unknown='ignore')

In [12]:
def enc_colnames(list_of_arrays, df_colnames):
    ldl = list(list_of_arrays)
    colnames = []
    for i in range(0, len(df_colnames)):
        if isinstance(ldl[i], str):
            lista = ldl
        else:
            lista = ldl[i]
        for j in range(0, len(lista)):
            name = df_colnames[i] + "_" + lista[j]
            colnames.append(name)
    return(colnames)

In [36]:
ecod_nonnum_colnames = enc_colnames(enc_nonnum.categories_, nonnum_df.columns)

In [37]:
ecod_nonnum_df = pd.DataFrame(enc_nonnum.transform(nonnum_df).toarray(), columns = ecod_nonnum_colnames)

In [38]:
ecod_nonnum_df.to_csv('encoded_case_c_0.csv', index=True)

2.4) Encoding numeric fields (Case B in report).

In [39]:
num_df = df[numeric_fields].astype(str)

In [40]:
enc_num = OneHotEncoder(handle_unknown='ignore')

In [41]:
enc_num.fit(num_df)

OneHotEncoder(handle_unknown='ignore')

In [42]:
ecod_num_colnames = enc_colnames(enc_num.categories_, num_df.columns)

In [43]:
ecod_num_df = pd.DataFrame(enc_num.transform(num_df).toarray(), columns = ecod_num_colnames)

In [44]:
ecod_num_df.to_csv('encoded_case_b_0.csv', index=True)

2.5) Encode the Rio marker labels.

In [45]:
enc_labels = OneHotEncoder(handle_unknown='ignore')

In [46]:
enc_labels.fit(labels_df)

OneHotEncoder(handle_unknown='ignore')

In [47]:
ecod_labels_colnames = enc_colnames(enc_labels.categories_, labels_df.columns)

In [48]:
ecod_labels_df = pd.DataFrame(enc_labels.transform(labels_df).toarray(), columns = ecod_labels_colnames)

In [49]:
ecod_labels_df.to_csv('encoded_labels.csv', index=True)

#### 3) Frequency vectorization

Encoding text fields (Case D)

In [6]:
determiners = "the, a, an, this, that, these, those, my, your, his, her, its, our, their, few, little, much, many, lot, most, some, any, enough, all, both, half, either, neither, each, every, other, another, such, what, rather, quite"
conjunctions = "and, or, but, yet, because, since, as, when, before, after, once, until, whenever, while, where, wherever, if, unless, case, although, though, whereas"
prepositions = "in, at, on, for, from, to, until, by, towards, across, through, between, among, of, with, by, about"
adverbs = "always, almost, usually, occasionally, regularly, seldom, never, rarely, hardly, often, generally, particularly, recently, immediately, usually, soon, usually, truly, ultimately, completely, mostly, perfectly"

In [7]:
det_list = determiners.split(", ")
conj_list = conjunctions.split(", ")
prep_list = prepositions.split(", ")
adv_list = adverbs.split(", ")
num_list = [str(i) for i in range(0,3000)]

In [8]:
all_words = det_list + conj_list + prep_list + num_list + adv_list + ["hello_world"]

In [9]:
vectorizer = CountVectorizer(input = "content", lowercase = "True", stop_words = all_words, analyzer = "word", min_df = 0.001)
title_desc_x = vectorizer.fit_transform(title_desc_en)

In [10]:
title_desc_mtx = list(title_desc_x.toarray())

In [13]:
title_desc_df_cols = enc_colnames(vectorizer.get_feature_names_out(), ["Text"])

In [14]:
title_desc_df = pd.DataFrame(title_desc_mtx, columns = title_desc_df_cols)

In [15]:
title_desc_df.head()

,Text_00,Text_000,Text_06,Text_089,Text_09,Text_1st,Text_2020intake,Text_2nd,Text_5m,Text_7th,...,Text_établissement,Text_états,Text_étude,Text_études,Text_été,Text_évaluation,Text_évaluations,Text_événements,Text_être,Text_œuvre
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
title_desc_df.to_csv('encoded_case_d_0.csv', index=True)